In [27]:
import logging
import datetime
import os
import json
from dotenv import load_dotenv

#for UI
import streamlit as st

# for function
from function import (connect_openai_llm, generate_answer, display_hits_dataframe)
# from prompt import generate_pdf_prompt

In [28]:
load_dotenv()
OPENAI_API_KEY=  os.getenv("OPENAI_API_KEY")


In [184]:
# json
# instruction = """You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the speficied subject and its content. From the email, please extract the following entity in to jason format provided below
# [{"Yacht Model": ""},{"Yacht Length": ""},{"Year of Manufacture":""}{"Current Value/Purchase Price":""},{"Current Location":""},{"Intended Cruising Area":""},{"Owner's Name":""},{"Owner's Contact Information":""},{"Owner's Boating Experience":""},{"Previous Insurance Claims":""},{"Additional Equipment":""},{"Current Insurance Coverage":""},{"Other":""}]"""

# pandas dataframe
instruction_dataframe = f"""You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the specified subject and its content. From the email, please extract the following entities into a list with the order specified below:
["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner's Name", "Owner's Contact Information", "Owner's Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]
If the information is not provide for the specific entities, that element of the list should be None. Return only in the string of list format that can be directly transfer into python list.
"""

# python dictionary
instruction_dic = """You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the speficied subject and its content. From the email, please extract the following entity in to python dictionary format provided below
{"Yacht Model": "","Yacht Length": "","Year of Manufacture": "","Current Value/Purchase Price": "","Current Location": "","Intended Cruising Area": "","Owner's Name": "","Owner's Contact Information": "","Owner's Boating Experience": "","Previous Insurance Claims": ","Additional Equipment": "","Current Insurance Coverage": "","Other": ""}
If the information is not provide for the specific entities, that value of the key should be None
"""

In [177]:
def generate_prompt(email):
    messages = {
        "messages": [
            {
            "content": instruction_dataframe,
            "role": "system"
            },
            {
            "content": f"""INSURANCE REQUEST EMAIL: {email}""",
            "role": "user"
            },
            {
            "content": "",
            "role": "assistant"
            },
        ]
    }
    return messages["messages"]

In [110]:
customer_email= """Subject: Insurance Quote for My New Sunseeker

To whom it may concern,

I recently purchased a Sunseeker Manhattan 68 yacht and I'm looking for comprehensive insurance coverage. The yacht is 68 feet long and was built in 2022. It's currently docked at Marina del Rey in California.

Some additional details:

- Purchase price: $3,200,000
- Max speed: 32 knots
- Sleeping capacity: 8 people
- Planning to use it mainly for coastal cruising, with occasional trips to Mexico

I have 10 years of boating experience, including 5 years with vessels over 50 feet. No accidents or insurance claims in the past.

Could you please provide me with a quote for full coverage? I'm particularly interested in options for liability coverage.

Thank you,
John Smith
Phone: (310) 555-1234"""

In [111]:
model_id_llm='gpt-4o'
model_id_emb="text-embedding-3-large"
model_llm = connect_openai_llm()

In [118]:
prompt = generate_prompt(customer_email)
print(prompt)
response = generate_answer(model_llm,prompt)
print(response)
st.text_area(label="Model Response", value=response, height=300)
# display_hits_dataframe(hits)

[{'content': 'You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the speficied subject and its content. From the email, please extract the following entity in to python dictionary format provided below\n{"Yacht Model": "","Yacht Length": "","Year of Manufacture": "","Current Value/Purchase Price": "","Current Location": "","Intended Cruising Area": "","Owner\'s Name": "","Owner\'s Contact Information": "","Owner\'s Boating Experience": "","Previous Insurance Claims": ","Additional Equipment": "","Current Insurance Coverage": "","Other": ""}\nIf the information is not provide for the specific entities, that value of the key should be None\n', 'role': 'system'}, {'content': "INSURANCE REQUEST EMAIL: Subject: Insurance Quote for My New Sunseeker\n\nTo whom it may concern,\n\nI recently purchased a Sunseeker Manhattan 68 yacht and I'm looking for comprehensive insurance coverage. The yacht is 68 feet long and was built in 2022. It's currently docked at 

'```python\n{\n  "Yacht Model": "Sunseeker Manhattan 68",\n  "Yacht Length": "68 feet",\n  "Year of Manufacture": "2022",\n  "Current Value/Purchase Price": "$3,200,000",\n  "Current Location": "Marina del Rey, California",\n  "Intended Cruising Area": "Coastal cruising, with occasional trips to Mexico",\n  "Owner\'s Name": "John Smith",\n  "Owner\'s Contact Information": "(310) 555-1234",\n  "Owner\'s Boating Experience": "10 years, including 5 years with vessels over 50 feet",\n  "Previous Insurance Claims": "No accidents or insurance claims in the past",\n  "Additional Equipment": None,\n  "Current Insurance Coverage": None,\n  "Other": "Interested in options for liability coverage"\n}\n```'

In [39]:
response_dic = response

In [50]:
response_json = response.replace('\n', '').replace('json', '').replace('```', '').replace('', '')
response_json

'[  {"Yacht Model": "Sunseeker Manhattan 68"},  {"Yacht Length": "68 feet"},  {"Year of Manufacture":"2022"},  {"Current Value/Purchase Price":"$3,200,000"},  {"Current Location":"Marina del Rey, California"},  {"Intended Cruising Area":"Coastal cruising, occasional trips to Mexico"},  {"Owner\'s Name":"John Smith"},  {"Owner\'s Contact Information":"Phone: (310) 555-1234"},  {"Owner\'s Boating Experience":"10 years of boating experience, including 5 years with vessels over 50 feet"},  {"Previous Insurance Claims":"No accidents or insurance claims in the past"},  {"Additional Equipment":""},  {"Current Insurance Coverage":""},  {"Other":"Max speed: 32 knots, Sleeping capacity: 8 people"}]'

In [52]:
data = json.loads(response_json)
data

[{'Yacht Model': 'Sunseeker Manhattan 68'},
 {'Yacht Length': '68 feet'},
 {'Year of Manufacture': '2022'},
 {'Current Value/Purchase Price': '$3,200,000'},
 {'Current Location': 'Marina del Rey, California'},
 {'Intended Cruising Area': 'Coastal cruising, occasional trips to Mexico'},
 {"Owner's Name": 'John Smith'},
 {"Owner's Contact Information": 'Phone: (310) 555-1234'},
 {"Owner's Boating Experience": '10 years of boating experience, including 5 years with vessels over 50 feet'},
 {'Previous Insurance Claims': 'No accidents or insurance claims in the past'},
 {'Additional Equipment': ''},
 {'Current Insurance Coverage': ''},
 {'Other': 'Max speed: 32 knots, Sleeping capacity: 8 people'}]

dataframe pd

In [56]:
import pandas as pd

In [64]:
col = ["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner's Name", "Owner's Contact Information", "Owner's Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]
df = pd.DataFrame(index=col)
df

""
Yacht Model
Yacht Length
Year of Manufacture
Current Value/Purchase Price
Current Location
Intended Cruising Area
Owner's Name
Owner's Contact Information
Owner's Boating Experience
Previous Insurance Claims


In [63]:
#str > list > Series > into df
import ast
response_list = ast.literal_eval(response)

In [65]:
df['new']=response_list

In [67]:
df

,new
Yacht Model,Sunseeker Manhattan 68
Yacht Length,68 feet
Year of Manufacture,2022
Current Value/Purchase Price,"$3,200,000"
Current Location,"Marina del Rey, California"
Intended Cruising Area,"Coastal cruising, with occasional trips to Mexico"
Owner's Name,John Smith
Owner's Contact Information,(310) 555-1234
Owner's Boating Experience,"10 years of boating experience, including 5 ye..."
Previous Insurance Claims,No accidents or insurance claims in the past


# Tavily search

In [69]:
from tavily import TavilyClient
apikey='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'
tavily_client = TavilyClient(api_key=apikey)

In [70]:
response = tavily_client.search("Who is Leo Messi?")
print(response)

{'query': 'Who is Leo Messi?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '33 Best Facts Of Messi', 'url': 'https://facts.net/lifestyle/sports/33-best-facts-of-messi/', 'content': "Messi's full name is Lionel Andrés Messi. Lionel Andrés Messi, widely known as Messi, was born on June 24, 1987, in Rosario, Argentina. His exceptional football skills have earned him global recognition and numerous accolades throughout his career. Messi joined FC Barcelona at the age of 13.", 'score': 0.9941801, 'raw_content': None}, {'title': 'Lionel Messi | Biography, Competitions, Wins and Medals', 'url': 'https://olympics.com/en/athletes/lionel-messi', 'content': '“The Olympic gold in 2008 is the win that I value the most because it is a tournament that you may play only once in your life and involves many athletes from different disciplines” - Lionel Messi, Spanish Esquire\nFeatured\nLatest News\nOlympic Results\nAthlete Olympic Results Content\nYou may like Lionel

In [79]:
type(response['results'])
response['results']

[{'title': '33 Best Facts Of Messi',
  'url': 'https://facts.net/lifestyle/sports/33-best-facts-of-messi/',
  'content': "Messi's full name is Lionel Andrés Messi. Lionel Andrés Messi, widely known as Messi, was born on June 24, 1987, in Rosario, Argentina. His exceptional football skills have earned him global recognition and numerous accolades throughout his career. Messi joined FC Barcelona at the age of 13.",
  'score': 0.9941801,
  'raw_content': None},
 {'title': 'Lionel Messi | Biography, Competitions, Wins and Medals',
  'url': 'https://olympics.com/en/athletes/lionel-messi',
  'content': '“The Olympic gold in 2008 is the win that I value the most because it is a tournament that you may play only once in your life and involves many athletes from different disciplines” - Lionel Messi, Spanish Esquire\nFeatured\nLatest News\nOlympic Results\nAthlete Olympic Results Content\nYou may like Lionel MESSI\nBiography\nBorn in Rosario, Argentina in 1987 Lionel Messi is widely held to be 

In [ ]:
email_John = """Subject: Insurance Quote for My New Sunseeker

To whom it may concern,

I recently purchased a Sunseeker Manhattan 68 yacht and I'm looking for comprehensive insurance coverage. The yacht is 68 feet long and was built in 2022. It's currently docked at Marina del Rey in California.

Some additional details:

- Purchase price: $3,200,000
- Max speed: 32 knots
- Sleeping capacity: 8 people
- Planning to use it mainly for coastal cruising, with occasional trips to Mexico

I have 10 years of boating experience, including 5 years with vessels over 50 feet. No accidents or insurance claims in the past.

Could you please provide me with a quote for full coverage? I'm particularly interested in options for liability coverage.

Thank you,
John Smith
Phone: (310) 555-1234"""

In [ ]:
email_Sarah = """Subject: Need insurance for secondhand yacht

Hi there,

I'm reaching out about insuring a yacht I'm considering buying. It's a used Azimut 50 Fly, I think from around 2018 or 2019. The current owner is asking $1.1 million.

I'm new to yacht ownership, but I've taken some courses in navigation and boat handling. The yacht will be kept at a marina in Miami most of the time, but I'd like the freedom to sail around the Caribbean.

Not sure what other information you need, but please let me know if you can give me a ballpark figure for insurance costs.

Thanks!
Sarah"""

In [80]:
import pandas as pd

# Load CSV file into a DataFrame
df = pd.read_csv('sample/result_extracted02-11-2024_2331.csv')

In [86]:
entities = ["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner's Name", "Owner's Contact Information", "Owner's Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]

In [89]:
df = df.drop(columns='tanyaton')
df.index =entities
df

,Sarah,John
Yacht Model,Azimut 50 Fly,Sunseeker Manhattan 68
Yacht Length,50,68 feet
Year of Manufacture,2018 or 2019,2022
Current Value/Purchase Price,$1.1 million,"$3,200,000"
Current Location,Miami,"Marina del Rey, California"
Intended Cruising Area,Caribbean,"Coastal cruising, trips to Mexico"
Owner's Name,Sarah,John Smith
Owner's Contact Information,Not provided,(310) 555-1234
Owner's Boating Experience,"New, has taken courses in navigation and boat ...","10 years, 5 years with vessels over 50 feet"
Previous Insurance Claims,Not mentioned,No accidents or claims


In [90]:
df

,Sarah,John
Yacht Model,Azimut 50 Fly,Sunseeker Manhattan 68
Yacht Length,50,68 feet
Year of Manufacture,2018 or 2019,2022
Current Value/Purchase Price,$1.1 million,"$3,200,000"
Current Location,Miami,"Marina del Rey, California"
Intended Cruising Area,Caribbean,"Coastal cruising, trips to Mexico"
Owner's Name,Sarah,John Smith
Owner's Contact Information,Not provided,(310) 555-1234
Owner's Boating Experience,"New, has taken courses in navigation and boat ...","10 years, 5 years with vessels over 50 feet"
Previous Insurance Claims,Not mentioned,No accidents or claims


In [ ]:
response_link = tavily_client.search(f"Search for Yacht model Azimut 50 Fly ")
response_link

{'query': "Search for ['Yacht Model', 'Yacht Length', 'Year of Manufacture'] of the Yacht model Azimut 50 Fly",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': '2022 Azimut 50 Flybridge 50 Boats for Sale - Edwards Yacht Sales',
   'url': 'https://www.edwardsyachtsales.com/boat/2022/azimut/50-flybridge/43329/',
   'content': 'View a large selection of Azimut boats for sale at Edwards Yacht Sales, the source of brokerage boats and yachts on the web. ... Manufacturer: Azimut Model: 50 Flybridge Year: 2022 Category: Power Condition: Used ... Hull Type: Hull Color: HIN: Designer: Flag of Registry: Dimensions & Weight. Length: 50 ft - 15.24 meter LOA: 50 ft - 15.24 meter',
   'score': 0.9998727,
   'raw_content': None},
  {'title': 'Azimut 50 Fly boats for sale - Boat Trader',
   'url': 'https://www.boattrader.com/boats/make-azimut/model-50-fly/',
   'content': 'Find 28 Azimut 50 Fly boats for sale near you, including boat prices, photos, and more. ... Mo

In [123]:
response['results'][0]['url']

TypeError: string indices must be integers

In [136]:
def generate_search_prompt(email_dic,link):
    messages = {
        "messages": [
            {
            "content": """given you a list of 3 internet link resources that contain the missing information for the given Yatch Model PYTHON DICTIONARY.
Please search through the links to find the information for the missing value (None) in the given PYTHON DICTIONARY. If all the informtion is found in the first link, there is no need to look in to the second or third link. If not, continue the search on the second link, and so on. 
Replace the None value in PYTHON DICTIONARY with new information and return only the Python Dictionary format below: 
{"Yacht Model": "","Yacht Length": "","Year of Manufacture": "","Current Value/Purchase Price": "","Current Location": "","Intended Cruising Area": "","Owner's Name": "","Owner's Contact Information": "","Owner's Boating Experience": "","Previous Insurance Claims": ","Additional Equipment": "","Current Insurance Coverage": "","Other": ""}
If you are unable to locate the missing information, simply return the Python dictionary with the values unchanged. Do not provide any further explanation
""",
            "role": "system"
            },
            {
            "content": f"""PYTHON DICTIONARY: {email_dic}
RESOURCE LINK: {link}""",
            "role": "user"
            },
            {
            "content": "",
            "role": "assistant"
            },
        ]
    }
    return messages["messages"]

In [127]:
response['results'][0]['url']

'https://www.edwardsyachtsales.com/boat/2019/azimut/50-flybridge/40356/'

In [193]:
# yatch_model = 'Yacht model Azimut 50 Fly'
email_dic = {
  "Yacht Model": "Sunseeker Manhattan 68",
  "Yacht Length": "68 feet",
  "Year of Manufacture": None,
  "Current Value/Purchase Price": None,
  "Current Location": "Marina del Rey, California",
  "Intended Cruising Area": "Coastal cruising, with occasional trips to Mexico",
  "Owner's Name": "John Smith",
  "Owner's Contact Information": "(310) 555-1234",
  "Owner's Boating Experience": "10 years, including 5 years with vessels over 50 feet",
  "Previous Insurance Claims": "No accidents or insurance claims in the past",
  "Additional Equipment": None,
  "Current Insurance Coverage": None,
  "Other": "Interested in options for liability coverage"
}

def search_web(email_dic, tavily_client):
    yatch_model = email_dic[0]
    message = f'Search for {entities[:3]} of the {yatch_model}'
    response = tavily_client.search(message)
    return [response['results'][0]['url'], response['results'][1]['url'], response['results'][2]['url']]


# link = search_web(email_dic, tavily_client, [:3])
# prompt = generate_search_prompt(email_dic,link)
# print(prompt)
# response = generate_answer(model_llm,prompt)
# print(response)

In [140]:
response

'{"Yacht Model": "Sunseeker Manhattan 68", "Yacht Length": "68 feet", "Year of Manufacture": "2025", "Current Value/Purchase Price": "$2,700,000", "Current Location": "Marina del Rey, California", "Intended Cruising Area": "Coastal cruising, with occasional trips to Mexico", "Owner\'s Name": "John Smith", "Owner\'s Contact Information": "(310) 555-1234", "Owner\'s Boating Experience": "10 years, including 5 years with vessels over 50 feet", "Previous Insurance Claims": "No accidents or insurance claims in the past", "Additional Equipment": "Jet skis, Seabobs", "Current Insurance Coverage": "Comprehensive hull and machinery, personal accident and liability", "Other": "Interested in options for liability coverage"}'

In [142]:
response_dic = json.loads(response)

In [145]:
response_dic.values()

dict_values(['Sunseeker Manhattan 68', '68 feet', '2025', '$2,700,000', 'Marina del Rey, California', 'Coastal cruising, with occasional trips to Mexico', 'John Smith', '(310) 555-1234', '10 years, including 5 years with vessels over 50 feet', 'No accidents or insurance claims in the past', 'Jet skis, Seabobs', 'Comprehensive hull and machinery, personal accident and liability', 'Interested in options for liability coverage'])

In [186]:
def response_format_dic(response):
    response_dic = response.replace('\n    ', '').replace('python', '').replace('```', '')
    data = json.loads(response_dic)
    return data

def response_format_df(response):
    response_list = ast.literal_eval(response)
    return response_list

In [171]:
def generate_prompt(email):
    messages = {
        "messages": [
            {
            "content": instruction_dic,
            "role": "system"
            },
            {
            "content": f"""INSURANCE REQUEST EMAIL: {email}""",
            "role": "user"
            },
            {
            "content": "{",
            "role": "assistant"
            },
        ]
    }
    return messages["messages"]

In [194]:
prompt = generate_prompt(customer_email)
email_dic = generate_answer(model_llm,prompt)
print('ppppppppp',email_dic)
# email_dic = ast.literal_eval(email_dic)

link = search_web(email_dic, tavily_client)
prompt_fill_na = generate_search_prompt(email_dic,link)
response = generate_answer(model_llm,prompt_fill_na)
response_dic = json.loads(response)
response_list = response_dic.values()


ppppppppp ["Sunseeker Manhattan 68", "68 feet", "2022", "$3,200,000", "Marina del Rey, California", "Coastal cruising, with occasional trips to Mexico", "John Smith", "(310) 555-1234", "10 years of boating experience, including 5 years with vessels over 50 feet", "No accidents or insurance claims in the past", None, None, None]


In [195]:
response_list

dict_values(['Sunseeker Manhattan 68', '68 feet', '2022', '$3,200,000', 'Marina del Rey, California', 'Coastal cruising, with occasional trips to Mexico', 'John Smith', '(310) 555-1234', '10 years of boating experience, including 5 years with vessels over 50 feet', 'No accidents or insurance claims in the past', '', '', ''])

In [157]:
email_dic = email_dic.replace('\n', '').replace('python', '').replace('```', '')

In [159]:
response_format_dic(str(email_dic))

JSONDecodeError: Expecting value: line 1 column 599 (char 598)

In [202]:
import json
import ast

entities = ["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner's Name", "Owner's Contact Information", "Owner's Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]

# instruction_json = """You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the speficied subject and its content. From the email, please extract the following entity in to jason format provided below
# [{"Yacht Model": ""},{"Yacht Length": ""},{"Year of Manufacture":""}{"Current Value/Purchase Price":""},{"Current Location":""},{"Intended Cruising Area":""},{"Owner's Name":""},{"Owner's Contact Information":""},{"Owner's Boating Experience":""},{"Previous Insurance Claims":""},{"Additional Equipment":""},{"Current Insurance Coverage":""},{"Other":""}]"""

instruction_dataframe = f"""You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the specified subject and its content. From the email, please extract the following entities into a list with the order specified below:
["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner's Name", "Owner's Contact Information", "Owner's Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]
If the information is not provide for the specific entities, that element of the list should be None. Return only in the string of list format that can be directly transfer into python list.
"""

# instruction_dic = """You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the speficied subject and its content. From the email, please extract the following entity in to python dictionary format provided below
# {"Yacht Model": "","Yacht Length": "","Year of Manufacture": "","Current Value/Purchase Price": "","Current Location": "","Intended Cruising Area": "","Owner's Name": "","Owner's Contact Information": "","Owner's Boating Experience": "","Previous Insurance Claims": ","Additional Equipment": "","Current Insurance Coverage": "","Other": ""}
# Answer in Dictionary format only. Do not provide any further explanation. If the information is not provide for the specific entities, that value of the key should be None.
# """

def generate_prompt(email):
    messages = {
        "messages": [
            {
            "content": instruction_dataframe,
            "role": "system"
            },
            {
            "content": f"""INSURANCE REQUEST EMAIL: {email}""",
            "role": "user"
            },
            {
            "content": "{",
            "role": "assistant"
            },
        ]
    }
    return messages["messages"]

# def response_format_json(response):
#     response_json = response.replace('\n', '').replace('json', '').replace('```', '').replace('', '')
#     data = json.loads(response_json)
#     return data

# def response_format_dic(response):
#     response_json = response.replace('\n', '').replace('python', '').replace('```', '').replace('', '')
#     data = json.loads(response_json)
#     return data

# def response_format_df(response):
#     response_list = ast.literal_eval(response)
#     return response_list

def search_web(email_dic, tavily_client):
    yatch_model = email_dic[0]
    message = f'Search for {entities[:3]} of the {yatch_model}'
    response = tavily_client.search(message)
    return [response['results'][0]['url'], response['results'][1]['url'], response['results'][2]['url']]


instruction_search_dic = """given you a list of 3 internet link resources that contain the missing information for the given Yatch Model PYTHON DICTIONARY.
Please search through the links to find the information for the missing value (None) in the given PYTHON DICTIONARY. If all the informtion is found in the first link, there is no need to look in to the second or third link. If not, continue the search on the second link, and so on. 
Replace the None value in PYTHON DICTIONARY with new information and return only the Python Dictionary format below: 
{"Yacht Model": "","Yacht Length": "","Year of Manufacture": "","Current Value/Purchase Price": "","Current Location": "","Intended Cruising Area": "","Owner's Name": "","Owner's Contact Information": "","Owner's Boating Experience": "","Previous Insurance Claims": ","Additional Equipment": "","Current Insurance Coverage": "","Other": ""}
If you are unable to locate the missing information, simply return the Python dictionary with the values unchanged. Do not provide any further explanation
"""

instruction_search_df = """You will be given a RESOURCE LINK and a PYTHON YATCH LIST. 
PYTHON YATCH LIST is a list of Yatch information in a list containing the following information respectively:
["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner's Name", "Owner's Contact Information", "Owner's Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]
RESOURCE LINK is a list of 3 internet links that contain the missing information for the given PYTHON YATCH LIST.

Please search through the links to find the information for the missing value (None) in the given PYTHON YATCH LIST. If all the informtion is found in the first link, there is no need to look in to the second or third link. If not, continue the search on the second link, and so on. 
Replace the None value in PYTHON YATCH LIST with new information and return only the Python List format. If you are unable to locate the missing information, simply return the Python list with the values unchanged. Do not provide any further explanation
Return result only in list format that can be directly transfer into python list.
"""

def generate_search_prompt(email_dic,link):
    messages = {
        "messages": [
            {
            "content": f"""{instruction_search_df}""",
            "role": "system"
            },
            {
            "content": f"""PYTHON DICTIONARY: {email_dic}
RESOURCE LINK: {link}""",
            "role": "user"
            },
            {
            "content": "",
            "role": "assistant"
            },
        ]
    }
    return messages["messages"]

In [203]:
customer_email = {
  "Yacht Model": "Sunseeker Manhattan 68",
  "Yacht Length": "68 feet",
  "Year of Manufacture": None,
  "Current Value/Purchase Price": None,
  "Current Location": "Marina del Rey, California",
  "Intended Cruising Area": "Coastal cruising, with occasional trips to Mexico",
  "Owner's Name": "John Smith",
  "Owner's Contact Information": "(310) 555-1234",
  "Owner's Boating Experience": "10 years, including 5 years with vessels over 50 feet",
  "Previous Insurance Claims": "No accidents or insurance claims in the past",
  "Additional Equipment": None,
  "Current Insurance Coverage": None,
  "Other": "Interested in options for liability coverage"
}

In [204]:
prompt = generate_prompt(customer_email)
email_dic = generate_answer(model_llm,prompt)
print('ppppppppp',email_dic)
# email_dic = ast.literal_eval(email_dic)

link = search_web(email_dic, tavily_client)
prompt_fill_na = generate_search_prompt(email_dic,link)
response = generate_answer(model_llm,prompt_fill_na)
print('response', response)
# response_dic = json.loads(response)
# response_list = response_dic.values()

ppppppppp ["Sunseeker Manhattan 68", "68 feet", None, None, "Marina del Rey, California", "Coastal cruising, with occasional trips to Mexico", "John Smith", "(310) 555-1234", "10 years, including 5 years with vessels over 50 feet", "No accidents or insurance claims in the past", None, None, "Interested in options for liability coverage"]
response ["Sunseeker Manhattan 68", "68 feet", "2021", "$3.5 million", "Marina del Rey, California", "Coastal cruising, with occasional trips to Mexico", "John Smith", "(310) 555-1234", "10 years, including 5 years with vessels over 50 feet", "No accidents or insurance claims in the past", "GPS, Autopilot, Solar Panels", "Full coverage with a deductible of $10,000", "Interested in options for liability coverage"]


In [201]:
type(response)

str